In [26]:
############################## can we decide between task related and MW ? ############################

# only two categorries

## Dependencies

# Import package/module for data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer



# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupKFold

# pipeline
#from sklearn.pipeline import Pipeline, FeatureUnion
from imblearn.pipeline import Pipeline

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# cross validation
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold

# balancing
from imblearn.over_sampling import SMOTE

#accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from IPython.display import display

from collections import Counter


In [27]:
read_path_awareness = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_label_awareness.csv"
read_path = r"W:\WCT\04_Mind-Wandering-Labstudy\04_Daten\04_Prepared_data\00_Julia\Model Building\features_with_labels.csv"
df = pd.read_csv(read_path)

#print(df.columns.tolist())

In [28]:
#df["Awareness_all"].unique()
df["Awareness_all"].value_counts()

task-related    631
aware           249
unaware         147
Name: Awareness_all, dtype: int64

In [29]:
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df["Awareness_all_new"] = labelencoder.fit_transform(df["Awareness_all"])
df["Awareness_all_new"].unique()

array([1, 0, 2])

In [30]:
df["Awareness_all_new"].value_counts()
# 1 = task related, 0 aware 2 unaware

1    631
0    249
2    147
Name: Awareness_all_new, dtype: int64

In [31]:
df["Awareness_all_new"] =  df["Awareness_all_new"].replace(2,0)
df["Awareness_all_new"].value_counts()
# 1 task related
# 0 MW

1    631
0    396
Name: Awareness_all_new, dtype: int64

In [32]:
df["Awareness_all_cat"].unique()
# to get only two categories
df["Awareness_all_cat"] =  df["Awareness_all_cat"].replace(3,2)
df["Awareness_all_cat"] =  df["Awareness_all_cat"].replace(1,0)
df["Awareness_all_cat"] =  df["Awareness_all_cat"].replace(2,1)
df["Awareness_all_cat"]

0       0
1       0
2       0
3       0
4       0
       ..
1022    1
1023    0
1024    0
1025    1
1026    0
Name: Awareness_all_cat, Length: 1027, dtype: int64

In [33]:
#nums = numpy.random.choice([0, 1], size=1000, p=[.1, .9])
# baseline for y_pred
baseline = np.ones(1027)
baseline[:396] = 0
np.random.shuffle(baseline)
baseline = baseline.astype(int)
baseline

array([1, 1, 0, ..., 1, 0, 0])

In [34]:
# ['task-related' == 1 , 'aware' == 2, 'unaware' == 3 , nan == delete the row]
def get_X_y(train):
    FEATURES = [
        'Fixation Duration Mean [ms]', 'Fixation Duration Max [ms]', 'Fixation Duration Min [ms]', 'Fixation Duration Median [ms]', 'Fixation Duration Std [ms]', 'Fixation Duration Skew [ms]', 'Fixation Duration Quantil 25 [ms]', 'Fixation Duration Quantil 75 [ms]',
        'Saccade Duration Mean [ms]', 'Saccade Duration Max [ms]', 'Saccade Duration Min [ms]', 'Saccade Duration Median [ms]', 'Saccade Duration Std [ms]', 'Saccade Duration Skew [ms]', 'Saccade Duration Quantil 25 [ms]', 'Saccade Duration Quantil 75 [ms]', 
        'Blink Duration Mean [ms]', 'Blink Duration Max [ms]', 'Blink Duration Min [ms]', 'Blink Duration Median [ms]', 'Blink Duration Std [ms]', 'Blink Duration Skew [ms]', 'Blink Duration Quantil 25 [ms]', 'Blink Duration Quantil 75 [ms]', 'Fixation Duration Kurtosis [ms]',
        'Saccade Duration Kurtosis [ms]',
        'Blink Duration Kurtosis [ms]', 
        'Fixation Saccade Ratio Mean', 'Fixation Saccade Ratio Max', 'Fixation Saccade Ratio Min', 'Fixation Saccade Ratio Median', 'Fixation Saccade Ratio Std', 'Fixation Saccade Ratio Skew', 'Fixation Saccade Ratio Kurtosis', 
        'Fixation Number', 'Blink Number', 
        'Fixation Dispersion X Mean [px]', 'Fixation Dispersion X Max [px]', 'Fixation Dispersion X Min [px]', 'Fixation Dispersion X Median [px]', 'Fixation Dispersion X Std [px]', 'Fixation Dispersion X Skew [px]', 'Fixation Dispersion X Quantil 25 [px]', 'Fixation Dispersion X Quantil 75 [px]', 
        'Fixation Dispersion Y Mean [px]', 'Fixation Dispersion Y Max [px]', 'Fixation Dispersion Y Min [px]', 'Fixation Dispersion Y Median [px]', 'Fixation Dispersion Y Std [px]', 'Fixation Dispersion Y Skew [px]', 'Fixation Dispersion Y Quantil 25 [px]', 'Fixation Dispersion Y Quantil 75 [px]', 'Fixation Dispersion X Kurtosis [px]', 'Fixation Dispersion Y Kurtosis [px]', 
        'Saccade Amplitude Mean [°]', 'Saccade Amplitude Max [°]', 'Saccade Amplitude Min [°]', 'Saccade Amplitude Median [°]', 'Saccade Amplitude Std [°]', 'Saccade Amplitude Skew [°]', 'Saccade Amplitude Quantil 25 [°]', 'Saccade Amplitude Quantil 75 [°]', 'Saccade Amplitude Kurtosis [°]',
        'Saccade Acceleration Average [°/s²] Mean', 'Saccade Acceleration Average [°/s²] Max', 'Saccade Acceleration Average [°/s²] Min', 'Saccade Acceleration Average [°/s²] Median', 'Saccade Acceleration Average [°/s²] Std', 'Saccade Acceleration Average [°/s²] Skew]', 'Saccade Acceleration Average [°/s²] Quantil 25]', 'Saccade Acceleration Average [°/s²] Quantil 75]',
        'Saccade Acceleration Peak [°/s²] Mean', 'Saccade Acceleration Peak [°/s²] Max', 'Saccade Acceleration Peak [°/s²] Min', 'Saccade Acceleration Peak [°/s²] Median', 'Saccade Acceleration Peak [°/s²] Std', 'Saccade Acceleration Peak [°/s²] Skew]', 'Saccade Acceleration Peak [°/s²] Quantil 25]', 'Saccade Acceleration Peak [°/s²] Quantil 75]', 'Saccade Deceleration Peak [°/s²] Mean', 
        'Saccade Deceleration Peak [°/s²] Max', 'Saccade Deceleration Peak [°/s²] Min', 'Saccade Deceleration Peak [°/s²] Median', 'Saccade Deceleration Peak [°/s²] Std', 'Saccade Deceleration Peak [°/s²] Skew]', 'Saccade Deceleration Peak [°/s²] Quantil 25]', 'Saccade Deceleration Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Average [°/s²] Mean', 'Saccade Velocity Average [°/s²] Max', 'Saccade Velocity Average [°/s²] Min', 'Saccade Velocity Average [°/s²] Median', 'Saccade Velocity Average [°/s²] Std', 'Saccade Velocity Average [°/s²] Skew]', 'Saccade Velocity Average [°/s²] Quantil 25]', 'Saccade Velocity Average [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [°/s²] Mean', 'Saccade Velocity Peak [°/s²] Max', 'Saccade Velocity Peak [°/s²] Min', 'Saccade Velocity Peak [°/s²] Median', 'Saccade Velocity Peak [°/s²] Std', 'Saccade Velocity Peak [°/s²] Skew]', 'Saccade Velocity Peak [°/s²] Quantil 25]', 'Saccade Velocity Peak [°/s²] Quantil 75]', 
        'Saccade Velocity Peak [%] Mean', 'Saccade Velocity Peak [%] Max', 'Saccade Velocity Peak [%] Min', 'Saccade Velocity Peak [%] Median', 'Saccade Velocity Peak [%] Std', 'Saccade Velocity Peak [%] Skew]', 'Saccade Velocity Peak [%] Quantil 25]', 'Saccade Velocity Peak [%] Quantil 75]', 
        'Saccade Acceleration Average [°/s²] Kurtosis', 'Saccade Acceleration Peak [°/s²] Kurtosis', 'Saccade Deceleration Peak [°/s²] Kurtosis', 'Saccade Velocity Average [°/s²] Kurtosis', 'Saccade Velocity Peak [°/s²] Kurtosis', 'Saccade Velocity Peak [%] Kurtosis', 
        'Saccade Length Mean [px]', 'Saccade Length Max [px]', 'Saccade Length Min [px]', 'Saccade Length Median [px]', 'Saccade Length Std [px]', 'Saccade Length Skew [px]]', 'Saccade Length Quantil 25 [px]]', 'Saccade Length Quantil 75 [px]]', 'Saccade Length Kurtosis [px]', 
        'Fixation Average Pupil Diameter [mm] Mean', 'Fixation Average Pupil Diameter [mm] Max', 'Fixation Average Pupil Diameter [mm] Min', 'Fixation Average Pupil Diameter [mm] Median', 'Fixation Average Pupil Diameter [mm] Std', 'Fixation Average Pupil Diameter [mm] Skew', 'Fixation Average Pupil Diameter [mm] Quantil25', 'Fixation Average Pupil Diameter [mm] Quantil75',
        'Fixation Average Pupil Diameter [mm] Kurtosis', 
        'Veregence Angles Mean [rad]', 'Veregence Angles Std [rad]', 
        'Pupil Distance Mean [px]', 'Pupil Distance Std [px]'
    ]

    GROUPS = "Participant"

    TARGET = "Awareness_all_new"

    X = train[FEATURES]
    y = train[TARGET]
    groups = train[GROUPS]
    return X, y, groups

In [35]:
X, y, groups = get_X_y(df)

In [36]:
##################### preprocessing with pipleline #####################
imputer =  SimpleImputer(fill_value='missing')
scaler = StandardScaler()
over = SMOTE(random_state= 27) #sampling_strategy=0.1

#model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
#model = GaussianNB()

#model = RandomForestClassifier(random_state=0)
model = SVC(kernel="rbf", C=0.025, probability=True)

steps = [('imputer', imputer), ('scaler',scaler), ('over', over), ('model', model)] #GaussianNB())## LinearSVC())
pipe = Pipeline(steps=steps)

In [37]:
##################### prediction without baseline #####################
import random
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
accurancy_scores = []
f1_scores = []
precision_scores = []
recall_scores = []

### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    accurancy_score = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    accurancy_scores.append(accurancy_score)
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)

    print(f"Accuracy for fold {fold}: ", accurancy_score)
    print(f"f1  score for fold {fold}: ", f1)
    print(f"recall for fold {fold}: ", recall)
    print(f"precision for fold {fold}: ", precision)
    
    fold += 1

mean_score = np.mean(accurancy_scores)
mean_f1 = np.mean(f1_scores)
mean_recall = np.mean(recall_scores)
mean_precision = np.mean(precision_scores)

print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
print(f'Our mean fold f1 score is {mean_f1:0.4f}')
print(f'Our mean fold recall is {mean_recall:0.4f}')
print(f'Our mean fold precisionis {mean_precision:0.4f}')


    0   1
0  36  42
1  32  99
Accuracy for fold 0:  0.645933014354067
f1  score for fold 0:  0.7279411764705883
recall for fold 0:  0.7557251908396947
precision for fold 0:  0.7021276595744681
    0   1
0  24  50
1  40  95
Accuracy for fold 1:  0.569377990430622
f1  score for fold 1:  0.6785714285714286
recall for fold 1:  0.7037037037037037
precision for fold 1:  0.6551724137931034
    0   1
0  30  45
1  53  82
Accuracy for fold 2:  0.5333333333333333
f1  score for fold 2:  0.6259541984732824
recall for fold 2:  0.6074074074074074
precision for fold 2:  0.6456692913385826
    0   1
0  36  46
1  30  94
Accuracy for fold 3:  0.6310679611650486
f1  score for fold 3:  0.7121212121212119
recall for fold 3:  0.7580645161290323
precision for fold 3:  0.6714285714285714
    0   1
0  49  38
1  39  67
Accuracy for fold 4:  0.6010362694300518
f1  score for fold 4:  0.6350710900473934
recall for fold 4:  0.6320754716981132
precision for fold 4:  0.638095238095238
Our mean fold accurancy_scores is

In [86]:
##################### baseline with predicted values #####################
import random
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
accurancy_scores = []
f1_scores = []
precision_scores = []
recall_scores = []

### stratifies group k fold
for train_index, test_index in sgk.split(X, y, groups):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    #print(f"Training target statistics: {Counter(y_train)}")
    #print(f"Testing target statistics: {Counter(y_test)}")
    
    #print(f"Training target statistics with oversampling: {Counter(y_res)}")
    #print(f"Testing target statistics with oversampling: {Counter(y_test)}")

    # Fit Model on Train
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    # baseline 
    
    baseline = random.sample(list(y_pred), len(y_pred))   
    # print(baseline)
    # print(y_pred)
    # y_baseline = [1] * len(y_pred)
    # print(len(y_baseline))
    # print(y_test.values)
    # unique, counts = np.unique(y_pred, return_counts=True)
    # print(dict(zip(unique, counts)))

    accurancy_score = accuracy_score(y_test, baseline)
    f1 = f1_score(y_test, baseline)
    precision = precision_score(y_test, baseline)
    recall = recall_score(y_test, baseline)

    accurancy_scores.append(accurancy_score)
    print(pd.DataFrame(confusion_matrix(y_test, baseline)))
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)


    # accurancy_score = accuracy_score(y_test, y_pred)
    # f1 = f1_score(y_test, y_pred)
    # precision = precision_score(y_test, y_pred)
    # recall = recall_score(y_test, y_pred)

    # accurancy_scores.append(accurancy_score)
    # print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    # f1_scores.append(f1)
    # precision_scores.append(precision)
    # recall_scores.append(recall)

    print(f"Accuracy for fold {fold}: ", accurancy_score)
    print(f"f1  score for fold {fold}: ", f1)
    print(f"recall for fold {fold}: ", recall)
    print(f"precision for fold {fold}: ", precision)
    
    fold += 1

mean_score = np.mean(accurancy_scores)
mean_f1 = np.mean(f1_scores)
mean_recall = np.mean(recall_scores)
mean_precision = np.mean(precision_scores)

print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
print(f'Our mean fold f1 score is {mean_f1:0.4f}')
print(f'Our mean fold recall is {mean_recall:0.4f}')
print(f'Our mean fold precisionis {mean_precision:0.4f}')


    0   1
0  33  45
1  37  94
Accuracy for fold 0:  0.6076555023923444
f1  score for fold 0:  0.6962962962962963
recall for fold 0:  0.7175572519083969
precision for fold 0:  0.6762589928057554
    0    1
0  19   55
1  34  101
Accuracy for fold 1:  0.5741626794258373
f1  score for fold 1:  0.6941580756013745
recall for fold 1:  0.7481481481481481
precision for fold 1:  0.6474358974358975
    0   1
0  31  44
1  49  86
Accuracy for fold 2:  0.5571428571428572
f1  score for fold 2:  0.6490566037735849
recall for fold 2:  0.6370370370370371
precision for fold 2:  0.6615384615384615
    0   1
0  18  64
1  29  95
Accuracy for fold 3:  0.5485436893203883
f1  score for fold 3:  0.6713780918727914
recall for fold 3:  0.7661290322580645
precision for fold 3:  0.5974842767295597
    0   1
0  30  57
1  48  58
Accuracy for fold 4:  0.45595854922279794
f1  score for fold 4:  0.5248868778280543
recall for fold 4:  0.5471698113207547
precision for fold 4:  0.5043478260869565
Our mean fold accurancy_sc

In [19]:
##################### Baseline with 38.% MW ##################### 
import random
sgk = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=529)

fold = 0
accurancy_scores = []
f1_scores = []
precision_scores = []
recall_scores = []

### stratifies group k fold
for i in range(1, 1000):
    for train_index, test_index in sgk.split(X, y, groups):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # Fit Model on Train
        pipe.fit(X_train, y_train)

        y_pred = pipe.predict(X_test)

        # create baseline 
        # 38.55891% von y_pred should be MW, MW = 0
        baseline = np.ones(len(y_pred))
        mw_size = 38.55891/100  * len(y_pred)
        baseline[:int(mw_size)] = 0
        np.random.shuffle(baseline)
        baseline = baseline.astype(int)
        
        accurancy_score = accuracy_score(y_test, baseline)
        f1 = f1_score(y_test, baseline)
        precision = precision_score(y_test, baseline)
        recall = recall_score(y_test, baseline)

        accurancy_scores.append(accurancy_score)
        print(pd.DataFrame(confusion_matrix(y_test, baseline)))
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)

        print(f"Accuracy for fold {fold}: ", accurancy_score)
        print(f"f1  score for fold {fold}: ", f1)
        print(f"recall for fold {fold}: ", recall)
        print(f"precision for fold {fold}: ", precision)
        
        fold += 1

print(f1_scores)
mean_score = np.mean(accurancy_scores)
mean_f1 = np.mean(f1_scores)
mean_recall = np.mean(recall_scores)
mean_precision = np.mean(precision_scores)

print(f'Our mean fold accurancy_scores is {mean_score:0.4f}')
print(f'Our mean fold f1 score is {mean_f1:0.4f}')
print(f'Our mean fold recall is {mean_recall:0.4f}')
print(f'Our mean fold precisionis {mean_precision:0.4f}')


    0   1
0  25  53
1  55  76
Accuracy for fold 0:  0.48325358851674644
f1  score for fold 0:  0.5846153846153846
recall for fold 0:  0.5801526717557252
precision for fold 0:  0.5891472868217055
    0   1
0  28  46
1  52  83
Accuracy for fold 1:  0.5311004784688995
f1  score for fold 1:  0.6287878787878788
recall for fold 1:  0.6148148148148148
precision for fold 1:  0.6434108527131783
    0   1
0  32  43
1  48  87
Accuracy for fold 2:  0.5666666666666667
f1  score for fold 2:  0.6566037735849056
recall for fold 2:  0.6444444444444445
precision for fold 2:  0.6692307692307692
    0   1
0  34  48
1  45  79
Accuracy for fold 3:  0.5485436893203883
f1  score for fold 3:  0.6294820717131475
recall for fold 3:  0.6370967741935484
precision for fold 3:  0.6220472440944882
    0   1
0  38  49
1  36  70
Accuracy for fold 4:  0.5595854922279793
f1  score for fold 4:  0.6222222222222222
recall for fold 4:  0.660377358490566
precision for fold 4:  0.5882352941176471
    0   1
0  31  47
1  49  82
